In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
<command-2232661055001628> in <cell line: 1>()
----> 1 import dlt
      2 from pyspark.sql.functions import *
      3 from pyspark.sql.types import *

/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch(name, globals, locals, fromlist, level)
    169             # Import the desired module. If you’re seeing this while debugging a failed import,
    170             # look at preceding stack frames for relevant error information.
--> 171             original_result = python_builtin_import(name, globals, locals, fromlist, level)
    172 
    173             is_root_import = thread_local._nest_level == 1

ModuleNotFoundError: No module named 'dlt'

In [0]:
import datetime
month = datetime.datetime.now().month

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS my_database

In [0]:
%sql
USE my_database

In [0]:
# File location and type
file_location = "s3://yadi-pipeline/month-{}/orders.csv".format(month)
file_type = "csv"
# temp_table_name = "orders"
# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
@dlt.table
def df_order():
    return (
        spark.read.format(file_type)
        .option("inferSchema", infer_schema)
        .option("header", first_row_is_header)
        .option("sep", delimiter)
        .load(file_location)
    )
    df_order.write.mode("append").saveAsTable('orders_table')

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-2168912468273097> in <cell line: 11>()
      9 
     10 # The applied options are for CSV files. For other file types, these will be ignored.
---> 11 @dlt.table
     12 def df_order():
     13     return (

NameError: name 'dlt' is not defined

In [0]:
#temp_table_name = "orders"
#@dlt.table
#def df_orders():
#    return dlt.read("df_order")\
#  .createOrReplaceTempView(temp_table_name)\
#  .write.format("hive").mode("append").saveAsTable('orders_table')

In [0]:
#temp_table_name = "orders"


#df_order.createOrReplaceTempView(temp_table_name)



In [0]:
# File location and type
file_location = "s3://yadi-pipeline/month-{}/products.csv".format(month)
file_type = "CSV"
temp_table_name = "products"
# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
@dlt.table
def df_products():
    return (
        spark.read.format(file_type)
        .option("inferSchema", infer_schema)
        .option("header", first_row_is_header)
        .option("sep", delimiter)
        .load(file_location)
        #.createOrReplaceTempView(temp_table_name)
        #.write.format("hive")
        #.mode("append")
        #.saveAsTable("products_table")
    )
    df_products.write.format("hive").mode("append").saveAsTable('products_table')

In [0]:
#temp_table_name = "products"

#df_products.createOrReplaceTempView(temp_table_name)


In [0]:
file_location = "s3://yadi-pipeline/month-{}/departments.csv".format(month)
file_type = "CSV"
temp_table_name = "departments"
# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
@dlt.table
def df_departments():
    return (
        spark.read.format(file_type)
        .option("inferSchema", infer_schema)
        .option("header", first_row_is_header)
        .option("sep", delimiter)
        .load(file_location)
    )  # .createOrReplaceTempView(temp_table_name)\

    df_departments.write.format("hive").mode("append").saveAsTable('departments_table')
# .write.format("hive").mode("append").saveAsTable('departments_table')

In [0]:
#temp_table_name = "departments"

#df_departments.createOrReplaceTempView(temp_table_name)

In [0]:
#df_departments.write.format("hive").mode("append").saveAsTable('departments_table')

In [0]:
file_location = "s3://yadi-pipeline/month-{}/aisles.csv".format(month)
file_type = "CSV"
temp_table_name = "aisles"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
@dlt.table
def df_aisles():
    return (
        spark.read.format(file_type)
        .option("inferSchema", infer_schema)
        .option("header", first_row_is_header)
        .option("sep", delimiter)
        .load(file_location)
    )  # .createOrReplaceTempView(temp_table_name)\
    df_aisles.write.format("hive").mode("append").saveAsTable('aisles_table')

# .write.format("hive").mode("append").saveAsTable('aisles_table')

In [0]:
#temp_table_name = "aisles"

#df_aisles.createOrReplaceTempView(temp_table_name)

In [0]:
#df_aisles.write.format("hive").mode("append").saveAsTable('aisles_table')

In [0]:
%sql CREATE OR REPLACE TABLE allproductsinfo_table AS (
  SELECT
    p.product_id,
    p.product_name,
    p.aisle_id,
    a.aisle,
    p.department_id
  FROM
    products_table p
    LEFT JOIN aisles_table a ON p.aisle_id = a.aisle_id
  ORDER BY
    p.product_id
)

In [0]:
%sql CREATE OR REPLACE TABLE pad_table AS(
  SELECT
    t.product_id,
    t.product_name,
    t.aisle_id,
    t.aisle,
    t.department_id,
    d.department
  FROM
    allproductsinfo_table t
    LEFT JOIN departments_table d on t.department_id = d.department_id
  ORDER BY
    t.product_id
)

In [0]:
@dlt.table
def df_pad():
    return (
        spark.sql("select * from pad_table")
        .withColumnRenamed("aisle", "aisle_name")
        .withColumnRenamed("department", "department_name")
    )
    df_pad.write.format("hive").mode("append").saveAsTable("pads_table")
    df_pad.write.format("csv").mode("append").save(
        "s3://yadi-pipeline/output/pad_table_month-{}".format(month)
    )

In [0]:
#@dlt.table
#def df_pad1 = df_pad.withColumnRenamed("aisle", "aisle_name")\
                #.withColumnRenamed("department", "department_name")
#df_pad = df_pad1
#df_pad.show(10)

In [0]:
#df_pads = df_pad.write.format("hive").mode("append").saveAsTable('pads_table')

In [0]:
%sql
DROP TABLE pad_table

In [0]:
#df_pads.write.format("csv").mode('append').save("s3://yadi-pipeline/output/pad_table_month-{}".format(month))

In [0]:
# File location and type
file_location = "s3://yadi-pipeline/month-{}/order_products__prior.csv".format(month)
file_type = "CSV"
temp_table_name = "prior"
# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
@dlt.table
def df_prior():
    return (
        spark.read.format(file_type)
        .option("inferSchema", infer_schema)
        .option("header", first_row_is_header)
        .option("sep", delimiter)
        .load(file_location)
    )
    df_prior.write.mode("append").saveAsTable('prior_table')

In [0]:
#temp_table_name = "prior"

#df_prior.createOrReplaceTempView(temp_table_name)

In [0]:
#df_prior.write.format("hive").mode("append").saveAsTable('prior_table')

In [0]:
%sql CREATE OR REFRESH LIVE TABLE op_table AS (
  SELECT
    p.order_id,
    o.user_id,
    p.product_id,
    p.add_to_cart_order,
    p.reordered
  FROM
    prior_table p
    LEFT JOIN orders_table o ON p.order_id = o.order_id
  ORDER BY
    p.order_id
)

In [0]:
%sql CREATE OR REFRESH LIVE TABLE opp_table AS (
  SELECT
    o.order_id,
    o.user_id,
    o.product_id,
    p.product_name,
    p.aisle_id,
    p.aisle,
    p.department_id,
    p.department,
    o.add_to_cart_order,
    o.reordered
  FROM
    LIVE.op_table o
    LEFT JOIN pad_table p ON o.product_id = p.product_id
  ORDER BY
    o.product_id
)

In [0]:
@dlt.table
def df_opp():
    return (
        spark.table("LIVE.opp_table")
        .withColumnRenamed("aisle", "aisle_name")
        .withColumnRenamed("department", "department_name")
    )
    df_opp.write.format("hive").mode("append").saveAsTable("opps_table")
    df_opp.write.format("csv").mode("append").save(
        "s3://yadi-pipeline/output/opps_table_month-{}".format(month)
    )

In [0]:
#df_opp1 = df_opp.withColumnRenamed("aisle", "aisle_name")\
                #.withColumnRenamed("department", "department_name")
#df_opp = df_opp1
#df_opp.show(10)

In [0]:
#df_opp.write.format("hive").mode("append").saveAsTable('opps_table')

In [0]:
%sql
DROP TABLE opp_table

In [0]:
#df_opps.write.format("csv").mode('append').save("s3://yadi-pipeline/output/opps_table_month-{}".format(month))

In [0]:
file_location = "s3://yadi-pipeline/month-{}/order_products__train.csv".format(month)
file_type = "CSV"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
@dlt.table
def df_train():
    return (
        spark.read.format(file_type)
        .option("inferSchema", infer_schema)
        .option("header", first_row_is_header)
        .option("sep", delimiter)
        .load(file_location)
    )


    df_train.write.format("hive").mode("append").saveAsTable("train_table")

In [0]:
#temp_table_name = "train"

#df_train.createOrReplaceTempView(temp_table_name)

In [0]:
#df_train.write.format("hive").mode("append").saveAsTable('train_table')

In [0]:
%sql
CREATE OR REPLACE TABLE ot_table AS (
  SELECT
    t.order_id,
    o.user_id,
    t.product_id,
    t.add_to_cart_order,
    t.reordered
  FROM
    train_table t
    LEFT JOIN orders_table o ON t.order_id = o.order_id
  ORDER BY
    t.order_id
)

In [0]:
%sql
CREATE OR REPLACE TABLE otp_table AS (
  SELECT
    o.order_id,
    o.user_id,
    o.product_id,
    p.product_name,
    p.aisle_id,
    p.aisle,
    p.department_id,
    p.department,
    o.add_to_cart_order,
    o.reordered
  FROM
    ot_table o
    LEFT JOIN pad_table p ON o.product_id = p.product_id
  ORDER BY
    o.product_id
)

In [0]:
@dlt.table
def df_otp():
    return (
        spark.sql("select * from otp_table")
        .withColumnRenamed("aisle", "aisle_name")
        .withColumnRenamed("department", "department_name")
    )


    df_otp.write.format("hive").mode("append").saveAsTable("otps_table")
    df_otp.write.format("csv").mode("append").save("s3://yadi-pipeline/output/otps_table_month-{}".format(month))

In [0]:
#df_otp1 = df_otp.withColumnRenamed("aisle", "aisle_name")\
                #.withColumnRenamed("department", "department_name")
#df_otp = df_otp1
#df_otp.show(10)

In [0]:
#df_otp.write.format("hive").mode("append").saveAsTable('otps_table')

In [0]:
%sql
DROP TABLE otp_table

In [0]:
#df_otps.write.format("csv").mode('append').save("s3://yadi-pipeline/output/otps_table_month-{}".format(month))